## Notes 

#### possible analyses

1. [ ] for each pair/day combination, fit a line to the p_change_percent vs v_net curve and determine the slope. Then, plot a scatter plot of that slope vs the total market volume over that day for multiple different PAIRS and day combinations, and color the scatter plot by the PAIR to see if there is clustering ? 

2. [ ] I am quite curious about why the FUTURES market is persistently priced very slightly higher in the btcusdt and ethusdt pairs. I wonder if this extends to all the pairs on the futures markets. Also... i am curious about the futures vs spot volume. I noticed that the BNB spot market has equivalent vs more volume than the futures. However for BTC the futures market is much larger. My initial thoughts are that BNB is an asset which has its own underlying blockchain and the BNB token can be used in a myriad of ways (saving on trading fees, staking, funding your binance dex account for example). Thus, people buying BNB are likely to actually WANT the underlying token and thus they have to use SPOT market. However, in the case of bitcoin, it is possible that people are not interested in using/moving the bitcoins and thus futures market is fine. Also, futures market has higher leverage. 

3. [ ] There could be a relationship between spot/futures concordance and spot/futures v_net. For btusdt_8_26_20 around 13.6 hrs we see the p_avg_diff (will have normalized measure soon) dip quite negative. After this dissociation continued for approximately 20-30 minutes there was a large increase in f_v_net (net volume in the futures market) characterized byroughly 3 spikes of large net buy volume over tens of minutes. This period was associated with an increase in the price of BTC from 11320 to 11460 (~ +1.2% change). Interestingly, it appears that the period of iitial decrease in fsc (futures spot correspondence) is associated with an increase in BTC price. Not sure what to think of this at this moment, will come back. 

4. [ ] Add the mark price to the futures/spot plots as a big STAR/ cross

5. [ ] Create a hashing and caching system for retrieving data. Discussion below. Plan to use tinydb for disk persistence (https://tinydb.readthedocs.io/en/stable/) 

Provide state_hash as a summary of the analysis workflow. This way when the analysis code changes the CACHE will retrieve the updated data. Unfortunately, this brings the problem of knowing if the analysis change even effects the particular data of interest. For now I can provide dummy state hash so that I do not needlessly re-read from disk when updating the source code. However, in the future I could consider adding something like "scopes" to the cache request. The cache function could check if the scopes of the analysis update and the cache request overlap. If they do not, then the cached value can be returned. However, there is an edge case where there are two updates, one within scope and then one outside the scope. When "main.fname" accesses the cache now, the most recent analysis hash will only have out of scope updates, however the previous one had within scope updates. Thus it is necessary to: 
1. keep track of the history of analysis hashes and the scopes (functions?) they changed 
2. keep track of the state of the analysis hash ('analysis_hash') when "main.fname" LAST accessed the cache function  

In order for the cache to be truly useful, it would ideally persist over kernel restarts. This means it must exist on disk. Although this adds more engineering complexity, it comes with the interesting advantage of being able to transfer the cache from one system to another, and thus immediately render relevant analyses. 

The access_id is meant to identify the accessing function. This provides namespaces so that cache requests do not get mixed up. May consider tree based strings for the access_id, for example "main.fn_name"

```
def check_cache(state_hash,access_id, access_params) : 
           unique_cache_key = (state_hash,access_id, access_params)  
           hit = CACHE[unique_cache_key]  
           if !hit :  
               print("cache miss!")
               result = DO_IO  
               CACHE[unique_cache_key] = result 
               return result  
           else :  
               print("cache hit!")  
               return hit 
```                
               
           
           

    

## Definitions for spot/futures analysis 

<b>s</b> - spot data <br/>
<b>f</b> - futures data <br/>

<b>s_p</b> = spot price <br/>
<b>f_p</b> = futures price <br/>
<b>m_p</b> = (f_p+s_p)/2 , [mean price]  <br/> 

<b>p_avg_diff</b> = f_p - s_p <br/>
<b>p_avg_diff_percent</b> =  p_avg_diff/m_p  <br/> 

The <b>p_avg_diff_percent</b> will provide a normalized measure (normalizable across the different PAIRS) of the "concordance" (i.e. percentage difference) of the spot and futures market. 





## Binance Info 

This is some reference info copied from binances website. 
First is the web socket message structure returned from the futures and spot 
websocket api. Fortunately, they follow the same structure. 

#### Description of data structure returned by binance futures and spot api:  
1. p: price of the trade 
2. q: quantity traded  (denominated in the "base" asset - asset1 in the pair asset1asset2) <br/>
    Source- per the binance documentation: 
    <b>"base asset refers to the asset that is the quantity of a symbol. For the symbol BTCUSDT, BTC would be the base asset.
quote asset refers to the asset that is the price of a symbol. For the symbol BTCUSDT, USDT would be the quote asset."</b> <br/>
    Thus, q and quantity traded are taken to be denominated in units of the FIRST asset in the pair ASSET1ASSET2.  
3. T: trade time in epoch time (ms) 
4. m: indicates if the trade was initiated by a buyer or seller. The exact wording, per binance's api is: 
    "Is the buyer the market maker?"<br/>
   Thus, when m==True, we infer that the trade was initiated by a SELLER, since the buyer was the market maker (and the buyer was the market maker) 


#### Example data responses ~>

### Binance Futures Return Value 
{<br>
  "e": "aggTrade",  // Event type<br>
  "E": 123456789,   // Event time<br>
  "s": "BTCUSDT",    // Symbol<br>
  "a": 5933014,     // Aggregate trade ID<br>
  "p": "0.001",     // Price<br>
  "q": "100",       // Quantity<br>
  "f": 100,         // First trade ID<br>
  "l": 105,         // Last trade ID<br>
  "T": 123456785,   // Trade time<br>
  "m": true,        // Is the buyer the market maker?<br>
}<br>

### Binance Spot Return Value 
{<br>
  "e": "aggTrade",  // Event type<br>
  "E": 123456789,   // Event time<br>
  "s": "BNBBTC",    // Symbol<br>
  "a": 12345,       // Aggregate trade ID<br>
  "p": "0.001",     // Price<br>
  "q": "100",       // Quantity<br>
  "f": 100,         // First trade ID<br>
  "l": 105,         // Last trade ID<br>
  "T": 123456785,   // Trade time<br>
  "m": true,        // Is the buyer the market maker?<br>
  "M": true         // Ignore<br>
}<br>

### REMEMBER:  
<b>As you can see, the field 'm' reflects if the buyer was the marker maker. I.e reflects if the trade was a market SELL  </b>